In [1]:
import pandas as pd

In [2]:
import json

In [3]:
df = pd.read_csv("raw_data/curated_gene_disease_associations_v7.0.tsv", sep="\t")

In [4]:
df.head()

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.000,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.000,2015.0,2015.0,1,0,CTD_human
2,2,A2M,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
3,2,A2M,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.000,2004.0,2019.0,1,0,CTD_human
4,2,A2M,0.529,0.769,C0009375,Colonic Neoplasms,group,C06;C04,Neoplastic Process,0.30,1.000,2004.0,2004.0,1,0,CTD_human


In [5]:
df.shape

(84038, 16)

In [6]:
data2018 = df[df['YearInitial'] <= 2018]

In [7]:
data2018.shape

(78472, 16)

In [8]:
output_df = pd.DataFrame()

output_df['Source node name'] = data2018['geneSymbol']

output_df['Target node name'] = data2018['diseaseId']

output_df['Source node type'] = 'gene'
output_df['Target node type'] = 'disease'
output_df['Relationship type'] = 'risk_gene'

In [9]:
output_df.to_csv("train/disgenet_v7.0_train_2018.csv")

In [10]:
#get differences i.e. new associations to validate

In [11]:
train = {x:set(data2018.loc[data2018['diseaseId'] == x, 'geneSymbol'].tolist()) for x in data2018['diseaseId'].unique()}


In [12]:
#use disease keys from the train data, diseases added since 2018 not useful
valid = {x:set(df.loc[df['diseaseId'] == x, 'geneSymbol'].tolist()) for x in data2018['diseaseId'].unique()}


In [13]:
list(valid.keys())[:10]

['C0019209',
 'C0036341',
 'C0002395',
 'C0007102',
 'C0009375',
 'C0011265',
 'C0011570',
 'C0011581',
 'C0019202',
 'C0022660']

In [14]:
has_new = {}
for disease in train.keys():
    diff = valid[disease].difference(train[disease])
    if len(diff) > 0:
        has_new[disease] = list(diff)

In [15]:
len(has_new.keys())

1140

In [16]:
data2018['diseaseId'].nunique()

10600

In [17]:
n_new = [len(x) for x in has_new.values()]

In [18]:
with open("test/disgenet_new_since_2018.txt",'w') as f:
    json.dump(has_new, f)